In [1]:
import torch
import pandas as pd
import pickle
import numpy as np
from collections import defaultdict

In [2]:
ratings = pd.read_pickle("../data/ml-20m-split/full.pkl")
ratings

,user,item,item_id,rating,timestamp
0,0,1,2,3.5,1112486027
1,0,28,29,3.5,1112484676
2,0,31,32,3.5,1112484819
3,0,46,47,3.5,1112484727
4,0,49,50,3.5,1112484580
...,...,...,...,...,...
20000258,138492,13754,68954,4.5,1258126920
20000259,138492,13862,69526,4.5,1259865108
20000260,138492,13875,69644,3.0,1260209457
20000261,138492,13993,70286,5.0,1258126944


In [7]:
EMBEDDING_DIM = 50
print("Loading GloVe embeddings...")
with open("../data/glove.twitter.27B/glove.twitter.27B.{}d.txt".format(EMBEDDING_DIM)) as glove_file:
    glove_embeddings = defaultdict(lambda: np.random.rand(EMBEDDING_DIM))
    i = 0
    for line in glove_file.readlines():
        values = line.split(" ")
        word = values[0]
        embedding = np.asarray(values[1:], dtype="float32")
        assert(len(embedding) == EMBEDDING_DIM)
        glove_embeddings[word] = embedding
        
        if i % 100000 == 0:
            print(i)
        i += 1

Loading GloVe embeddings...
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
